# Capstone Project - The Battle of Neighborhoods

## Introduction/Business Problem

For this work I choose 2 biggest cities of my native country: Russia. This cities are Saint Petersburg (SPb) and Moscow. For all my live I have heart about differences between this two cities. Sometimes It seems that SPb and Moscow are differ in all aspects. And people, who lived in SPb and Moscow also differs: we have various habbits, favorite places, and even named some things in different ways. 

To tell the truth I would not like to compare cities wholly. Because all cities have "Sleep districts" and generally all of them are very common. It is much more interesting to compare centers of the cities, exactly Sights (showplace) surroundings - all that tourists can see then go to museums or view memorials

Business problem? It could be anything.
1 When travel agencies make sightseeing tours, usually tourists have time to walk or have a lunch during tour - So choosing the best place for it - where each person could find something for whom taste and wallet
2 Places near Sights are usually good gift shops, restaurants and many other places, so for Businessmens who want to open shop or smtg else it could be important to know place surroundings and compare different surroundings
3 I know that many trading network from Moscow are failed to become successful in SPB and vice versa. But also There are some successful examples. And noone know about reasons of it. Comparison of centers of two cities could be useful for buisenessmens who successful in one city and want to expand theis buisness to another

## Data

Really find information about Sights and its location is easy. Just ask google or another search sistem about Sights of cities and it give you million results. To coolect all data together or find place, where will be all data we need - It is our task
Fortunatly, on Foursquare we can find all we need, so I decided just ask it for search museums of SPb and Moscow

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
address_spb = 'Saint-Petersburg'

geolocator = Nominatim(user_agent="foursquare_agent")
location_spb = geolocator.geocode(address_spb)
latitude_spb = location_spb.latitude
longitude_spb = location_spb.longitude
print(latitude_spb, longitude_spb)

59.938732 30.316229


In [4]:
museum_spb = 'Museum'
radius = 5000
print(museum_spb + ' .... OK!')

Museum .... OK!


In [15]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius

In [16]:
url_spb = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_spb, longitude_spb, VERSION, museum_spb, radius, LIMIT)
results_spb = requests.get(url_spb).json()

In [17]:
# assign relevant part of JSON to venues
venues_spb = results_spb['response']['venues']

# tranform venues into a dataframe
df_spb = json_normalize(venues_spb)
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_spb.columns if col.startswith('location.')] + ['id']
df_spb_filtered = df_spb.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_spb_filtered['categories'] = df_spb_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_spb_filtered.columns = [column.split('.')[-1] for column in df_spb_filtered.columns]
df_spb_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Hermitage Museum (Государственный Эрмитаж),Art Museum,"Дворцовая наб., 34",RU,Санкт-Петербург,Россия,Дворцовая пл.,305,"[Дворцовая наб., 34 (Дворцовая пл.), 191186, С...","[{'lng': 30.313721, 'lat': 59.941173, 'label':...",59.941173,30.313721,NaN,191186,Санкт-Петербург,4d9abff1422ea1cd52d9ec4c
1,Russian Museum (Русский музей),Art Museum,"Инженерная ул., 4",RU,Санкт-Петербург,Россия,NaN,897,"[Инженерная ул., 4, 191011, Санкт-Петербург, Р...","[{'lng': 30.332318544387817, 'lat': 59.9387170...",59.938717,30.332319,NaN,191011,Санкт-Петербург,4bbc64cd51b89c742155872a
2,Museum of Soviet Arcade Machines (Музей советс...,Arcade,"Конюшенная пл., 2, лит. В",RU,Санкт-Петербург,Россия,NaN,619,"[Конюшенная пл., 2, лит. В, Санкт-Петербург, Р...","[{'lng': 30.326908298632507, 'lat': 59.9402810...",59.940281,30.326908,NaN,NaN,Санкт-Петербург,519690dd498e82098f5f0701
3,Russian Museum (Benois Wing) (Русский музей (К...,Art Gallery,"наб. Канала Грибоедова, 2",RU,Санкт-Петербург,Россия,Инженерная ул.,703,"[наб. Канала Грибоедова, 2 (Инженерная ул.), 1...","[{'lng': 30.328853130340576, 'lat': 59.9386611...",59.938661,30.328853,NaN,191011,Санкт-Петербург,4dbd3e31fa8cee727361cfae
4,Fabergé Museum (Музей Фаберже),Art Museum,"наб. реки Фонтанки, 21",RU,Санкт-Петербург,Россия,NaN,1563,"[наб. реки Фонтанки, 21, 191023, Санкт-Петербу...","[{'lng': 30.34313856863994, 'lat': 59.93480234...",59.934802,30.343139,NaN,191023,Санкт-Петербург,528b1a9611d20cbd10276bb2


In [18]:
df_spb_filtered.shape

(50, 16)

So we have location of 50 museums in SPb
and now lets make all the same for Moscow

In [10]:
address_msc = 'Moscow'

geolocator = Nominatim(user_agent="foursquare_agent")
location_msc = geolocator.geocode(address_msc)
latitude_msc = location_msc.latitude
longitude_msc = location_msc.longitude
print(latitude_msc, longitude_msc)

55.7504461 37.6174943


In [11]:
museum_msc = 'Museum'
print(museum_spb + ' .... OK!')

Museum .... OK!


In [12]:
url_msc = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_msc, longitude_msc, VERSION, museum_msc, radius, LIMIT)
results_msc = requests.get(url_msc).json()

In [13]:
# assign relevant part of JSON to venues
venues_msc = results_msc['response']['venues']

# tranform venues into a dataframe
df_msc = json_normalize(venues_msc)
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_spb.columns if col.startswith('location.')] + ['id']
df_msc_filtered = df_msc.loc[:, filtered_columns]

# filter the category for each row
df_msc_filtered['categories'] = df_msc_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_msc_filtered.columns = [column.split('.')[-1] for column in df_msc_filtered.columns]
df_msc_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,The Pushkin State Museum of Fine Arts (Государ...,Art Museum,"ул. Волхонка, 12",RU,Москва,Россия,Колымажный пер.,832,"[ул. Волхонка, 12 (Колымажный пер.), 119019, М...","[{'lng': 37.60548934223968, 'lat': 55.74723790...",55.747238,37.605489,NaN,119019,Москва,4b5ac73cf964a52037d428e3
1,The State Historical Museum (Государственный и...,History Museum,"Красная пл., 1",RU,Москва,Россия,пр. Воскресенские Ворота,544,"[Красная пл., 1 (пр. Воскресенские Ворота), 10...","[{'lng': 37.61776685714721, 'lat': 55.75533068...",55.755331,37.617767,Красная площадь,109012,Москва,4ba4b8def964a52024af38e3
2,Moscow Museum of Modern Art (Московский музей ...,Art Museum,"Гоголевский бул., 10",RU,Москва,Россия,NaN,1148,"[Гоголевский бул., 10, 119019, Москва, Россия]","[{'lng': 37.60026276111603, 'lat': 55.74693418...",55.746934,37.600263,NaN,119019,Москва,4baf14b5f964a52049ea3be3
3,Moscow Museum of Modern Art (Московский музей ...,Art Museum,"ул. Петровка, 25",RU,Москва,Россия,NaN,1874,"[ул. Петровка, 25, 107031, Москва, Россия]","[{'lng': 37.61395114204806, 'lat': 55.76716254...",55.767163,37.613951,NaN,107031,Москва,4beec007e8c3c928bb0e9992
4,Garage Museum of Contemporary Art (Музей совре...,Art Museum,ЦПКиО им. Горького,RU,Москва,Россия,"ул. Крымский Вал, 9, стр. 32",2716,"[ЦПКиО им. Горького (ул. Крымский Вал, 9, стр....","[{'lng': 37.60160618970004, 'lat': 55.72774207...",55.727742,37.601606,NaN,119049,Москва,4d624fdb29105941fffc0847


In [14]:
df_msc_filtered.shape

(50, 16)

and 50 museums of Moscow, thats it: all data we need are in two dataframes: df_spb_filtered and df_msc_filtered